# Imports

In [1]:
import os
import pandas as pd

import xlsxwriter
import openpyxl

from google.colab import drive

In [2]:
openpyxl.__version__

'3.1.2'

In [3]:
pd.__version__

'2.2.1'

In [4]:
# ! pip install xlsxwriter==3.2.0

In [5]:
# ! pip install pandas==2.2.1

In [6]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# from helpers import create_dummy_xslx_file

# Create Dummy Excel File

In [8]:
# create_dummy_xslx_file()

# GLOBAL_VARS

In [9]:
# CWD = os.getcwd()

CWD = "/content/drive/MyDrive/"
PATH_TO_INDEX = os.path.join(CWD, "folders")

FOLDERS = ["folder_1", "folder_2", "folder_3"]
EXCEL_FILE_PATH = os.path.join(CWD, "agg.xlsx")
EXCEL_FILE_PATH_TEMP = os.path.join(CWD, "agg_.xlsx")

# Helpers

In [10]:
def read_excel_specific_sheet(excel_path, excel_sheet_name):

    df = pd.read_excel(excel_path, sheet_name = excel_sheet_name)

    return df


def index_files(path, stored_files = []):

    folder_name = path.split(os.path.sep)[-1]
    files = [file for file in os.listdir(path = path) if file not in stored_files]
    df = pd.DataFrame(files).rename(columns = {0:folder_name})

    return df


def get_all_files_to_write(files_in_excel, files_indexed, column_name):
    return pd.DataFrame(data = files_in_excel + files_indexed, columns = column_name)


def main(path, folder_name, excel_file_path):

    files_in_excel = read_excel_specific_sheet(excel_file_path, excel_sheet_name=folder_name)

    files_in_excel = files_in_excel[folder_name].values.tolist()

    files_indexed = index_files(path = path, stored_files = files_in_excel)

    if files_indexed.shape[0] == 0:
      files_indexed = []
    else:
      files_indexed = files_indexed[folder_name].values.tolist()

    files_to_write = get_all_files_to_write(
        files_in_excel = files_in_excel,
        files_indexed = files_indexed,
        column_name = [folder_name]

    )

    return files_to_write


def final_cleanup():
    os.remove(EXCEL_FILE_PATH)
    os.rename(EXCEL_FILE_PATH_TEMP, EXCEL_FILE_PATH)

In [11]:
files_in_excel = read_excel_specific_sheet(EXCEL_FILE_PATH, excel_sheet_name="folder_2")

In [12]:
files_in_excel

,Unnamed: 0,folder_2
0,0,3
1,1,4
2,2,5
3,3,file6.pdf
4,4,file4.pdf
5,5,file5.pdf


In [13]:
files_in_excel

,Unnamed: 0,folder_2
0,0,3
1,1,4
2,2,5
3,3,file6.pdf
4,4,file4.pdf
5,5,file5.pdf


In [14]:
index_files(path = os.path.join(PATH_TO_INDEX, "folder_1"))

,folder_1
0,file4.pdf
1,file1.pdf
2,file3.pdf
3,file2.pdf
4,Copia de file2.pdf
5,Copia de file1.pdf
6,Copia de file4.pdf
7,Copia de file3.pdf


In [15]:
os.path.isfile(EXCEL_FILE_PATH)

True

In [16]:
os.path.isdir(os.path.join(PATH_TO_INDEX, "folder_2"))

True

In [17]:
EXCEL_FILE_PATH_TEMP

'/content/drive/MyDrive/agg_.xlsx'

In [18]:
with pd.ExcelWriter(EXCEL_FILE_PATH_TEMP) as writer:
  pass

In [19]:
with pd.ExcelWriter(EXCEL_FILE_PATH_TEMP) as writer:

    for folder_name in ["folder_1", "folder_2", "folder_3"]:

        path = os.path.join(PATH_TO_INDEX, folder_name)

        file_to_write = main(
            path = path,
            folder_name = folder_name,
            excel_file_path = EXCEL_FILE_PATH
        )

        file_to_write.to_excel(writer, sheet_name = folder_name)

In [20]:
final_cleanup()